In [1]:
%matplotlib inline
from jupyter_mplsettings import *
from stack_util import local_import
matplotlib.rc('font', **font)
from ipywidgets import widgets
from ipywidgets import *
from IPython.display import display, clear_output
from stack_analyze import PrepareModels, AnalyzePosts

Using TensorFlow backend.


In [2]:
cfg = local_import("./livedemo.py")
cfg = PrepareModels(cfg)

Loading word dictionary...
Preparing keras_tagprediction.
Preparing keras_scoreprediction_twoclasses_cnn.


In [3]:
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

button = widgets.Button(description='Submit')
titlefield = widgets.Text(value="my bad title", width=700)
postfield = widgets.Textarea(value='type your question...', width=500)
out = widgets.Output()

form_items = [
    Box([Label(value='Title'), titlefield], 
        layout=form_item_layout),
    Box([Label(value='Question'), postfield], 
        layout=form_item_layout),
    Box([Label(value=''),
        button], 
        layout=form_item_layout),
    Box([Label(value=''),
         out],
        layout=form_item_layout)
]

def on_submit(b):
    userpost = {"Body": postfield.value,
                "Title": titlefield.value,
                "CreationDate": "27/11/2017",
                "UserName": "testuser"}
    res = AnalyzePosts(cfg, userposts=[userpost], pids=None, debug=False)
    
    tagprobs = res["keras_tagprediction"][0]
    goodprobs = res["keras_scoreprediction_twoclasses_cnn"][0]
    
    prob_sort = np.argsort(tagprobs)[::-1][:3]
    probs = tagprobs[prob_sort]
    tags = cfg.mostcommon_tags.tags.values
    tags[30] = "other / not identified"
    prob_sort_topics = tags[prob_sort]
    
    with out:
        clear_output()
        print "Three hottest recognized topics:"
        for ip in range(len(prob_sort)):
            print "%s (%.1f percent)" % (prob_sort_topics[ip], 100. * probs[ip])
        if goodprobs[1] > goodprobs[0]:
            print "We estimate that this question is more likely a good question."
        else:
            print "We estimate that this question is more likely not very good. Have a look if you can provide more information or structure your question better. Also check out the suggestions below."
        print "Estimated probability to be a good question: %.1f percent" % (goodprobs[1] * 100.)
    
        print "\nSuggestions based on simple text analysis:"
        pfeatures = cfg.data["post_features"]
        feat_qs = cfg.feat_quantiles
        feat_qs = feat_qs.reset_index()
        features = feat_qs.feature.values
        feat_qs.set_index("feature", inplace=True)
        evaluation = ["extremely low", "very low", "low", "low to moderate", "moderate to high", "high", "very high", "extremely high"]
        desc = {"BodyNCodes": "Number of code fields", "BodyNQMarks": "Number of question marks",
                "BodySize": "Size of the post", "nwords": "Number of words",
                "ordermean": "Word prevalence", "ratio": "Ratio of verbs to nouns",
                "titlelen": "Length of the title"}
        for feat in features:
            if feat == "quants":
                continue
            qs = feat_qs.loc[feat].values
            val = pfeatures[feat][0]
#             loc = np.digitize(val, qs)
            loc = np.searchsorted(qs, val)
            if feat in desc:
                print "%s: %s" % (desc[feat], evaluation[loc])
            
    
    
button.on_click(on_submit)

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    width='50%'
))
form

Box(children=(Box(children=(Label(value=u'Title'), Text(value=u'my bad title')), layout=Layout(display=u'flex', flex_flow=u'row', justify_content=u'space-between')), Box(children=(Label(value=u'Question'), Textarea(value=u'type your question...')), layout=Layout(display=u'flex', flex_flow=u'row', justify_content=u'space-between')), Box(children=(Label(value=u''), Button(description=u'Submit', style=ButtonStyle())), layout=Layout(display=u'flex', flex_flow=u'row', justify_content=u'space-between')), Box(children=(Label(value=u''), Output()), layout=Layout(display=u'flex', flex_flow=u'row', justify_content=u'space-between'))), layout=Layout(align_items=u'stretch', border=u'solid 1px', display=u'flex', flex_flow=u'column', width=u'50%'))